In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

from cvt_utils import SpaceIterMesh, plot_cell_size_and_density, plot_seeds_and_cells, plot_bound_paths, display_animation

plt.style.use("dark_background")
np.random.seed(21)

# Problem definition
x_min = 0.0
x_max = 1.0
n_cells = 60
# n_cells = 100
# n_iters = 300
# n_iters = 2000
n_iters = 7000
# n_iters = 10000

mesh = SpaceIterMesh(x_min, x_max, n_cells, n_iters)
mesh.cell_size = lambda x: 0.1 + 0.0 * x
# mesh.cell_size = lambda x: np.exp(2*x)
# mesh.cell_size = lambda x: 1.0 + 0.5 * np.sin(2 * np.pi * x)

In [ ]:
plot_cell_size_and_density(mesh)

### Iterations

In [ ]:
# Generate random x values for the first iteration, sort them, and compute cell boundaries
mesh.seed_matrix[:, 0] = np.sort(np.random.uniform(x_min, x_max, n_cells))

for iter in range(n_iters):
    mesh.update_cell_bounds(iter)
    if iter < n_iters - 1:
        mesh.update_cell_seeds(iter)

In [ ]:
html = display_animation(mesh, duration=3.0, fps=60)
html

In [ ]:
plot_bound_paths(mesh, 0, n_iters - 1)

### Errors

In [ ]:
analytical = np.linspace(x_min, x_max, n_cells + 1)
min_errors = np.zeros(n_iters)
max_errors = np.zeros(n_iters)
rms_errors = np.zeros(n_iters)

for iter in range(n_iters):
    errors = mesh.bound_matrix[:, iter] - analytical
    min_errors[iter] = np.min(np.abs(errors))
    max_errors[iter] = np.max(np.abs(errors))
    rms_errors[iter] = np.sqrt(np.mean(errors**2))

In [ ]:
from cvt_utils import plot_errors

In [ ]:
plot_errors(min_errors, max_errors, rms_errors, logscale=False)
plot_errors(min_errors, max_errors, rms_errors, logscale=True)

### Convergence

In [ ]:
from cvt_utils import plot_log_errors

plot_log_errors(min_errors, max_errors, rms_errors)

slope, intercept = np.polyfit(np.log(rms_errors[:-1]), np.log(rms_errors[1:]), 1)
print(f"Observed order of convergence q ≈ {round(slope, 5)}")

### Energy